<a href="https://colab.research.google.com/github/cshengmin/Stockbuylowsellhigh/blob/main/my_best_daily_option_analysis_4_30_newest_so_far.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 📊 Daily Portfolio Tracker — Enhanced

!pip install yfinance yahooquery --quiet

import yfinance as yf
from yahooquery import Ticker
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Define stocks and ETFs separately

stock_tickers = [
    "NVDA", "META", "MU", "AMZN", "CRM", "MSFT", "GOOGL", "AVGO", "NOW",
    "AAPL", "WMT", "HD", "PLTR", "TSM", "APP", "HIMS", "CRWD"
]

etf_tickers = [
    "QQQ", "QQQM", "IVV", "SPLG", "TQQQ", "SPYU", "SCHG", "SPY", "IAU","SOXL"
]

# stock_tickers = [
#     "AAPL", "AMD", "AMZN", "APP", "ARM", "ASML", "AVGO", "BDX", "C", "CHWY",
#     "COIN", "CRM", "CRWD", "CVX", "CYBR", "DAL", "DIS", "DJT", "ENTG", "EVER",
#     "GE", "GOOGL", "GS", "HD", "HES", "HIMS", "HOOD", "ICHR", "INTC", "JPM",
#     "META", "MSFT", "MU", "NOW", "NVDA", "PLTR", "ROKU", "RXRX", "SERV", "SOFI",
#     "SOUN", "SPOT", "SU", "TSLA", "TSM", "UBER", "UNH", "VKTX", "VRSN", "VST",
#     "WMT", "WRD"
# ]


# etf_tickers = [
#     "ICOP", "ILIT", "ITOT", "IVV", "QQQ", "QQQM", "REZ", "SCHF",
#     "SCHG", "SPLG", "VUG", "XLC", "XLF", "XLG"
# ]


all_tickers = stock_tickers + etf_tickers

# Historical dates
dates_to_pull = ["2025-03-19", "2025-03-26", "2025-04-04"]

# Download historical data
hist_prices = yf.download(all_tickers, start="2025-03-18", end="2025-04-05", group_by="ticker", auto_adjust=True)

# Closing price snapshots
def get_price_on_date(ticker, date):
    try:
        return round(hist_prices[ticker]["Close"].loc[date], 2)
    except:
        return np.nan

price_snapshots = {t: " \\ ".join([str(get_price_on_date(t, d)) for d in dates_to_pull]) for t in all_tickers}

# 10-Day Moving Avg
end = datetime.today()
start = end - timedelta(days=15)
hist = yf.download(all_tickers, start=start, end=end, group_by="ticker", auto_adjust=True)

dma_10 = {t: hist[t]["Close"].rolling(10).mean().iloc[-1] if t in hist else np.nan for t in all_tickers}

# Current price and 52-week range
yf_data = yf.Tickers(" ".join(all_tickers))
current_prices, week_52 = {}, {}
for t in all_tickers:
    info = yf_data.tickers[t].info
    current_prices[t] = info.get("regularMarketPrice", np.nan)
    week_52[t] = f'{info.get("fiftyTwoWeekLow", "N/A")} - {info.get("fiftyTwoWeekHigh", "N/A")}'

# Analyst rating and target price
yq = Ticker(all_tickers)
financial_data = yq.financial_data
analyst_rating, target_price = {}, {}

for t in all_tickers:
    try:
        rating = financial_data[t]["recommendationMean"]
        analyst_rating[t] = (
            "Strong Buy" if rating <= 2 else
            "Buy" if rating <= 2.5 else
            "Hold" if rating <= 3 else
            "Sell" if rating <= 4 else "N/A"
        )
    except:
        analyst_rating[t] = "N/A"

    try:
        target_price[t] = financial_data[t]["targetMeanPrice"]
    except:
        target_price[t] = np.nan

# Supplement missing ETF target prices
etf_target_prices = {
    "QQQ": 0, "QQQM": 0, "IVV": 0, "SPLG": 0,
    "TQQQ": 0, "SPYU": 0, "SCHG": 0, "SPY": 0, "IAU": 0, "SOXL": 0
}
for t in etf_tickers:
    if pd.isna(target_price[t]) and etf_target_prices.get(t):
        target_price[t] = etf_target_prices[t]

# Discount/Premium calculation
discount_premium = {}
for t in all_tickers:
    diffs = []
    for d in dates_to_pull:
        try:
            past_price = hist_prices[t]["Close"].loc[d]
            curr_price = current_prices[t]
            if pd.notna(past_price) and pd.notna(curr_price):
                pct_diff = ((curr_price - past_price) / past_price) * 100
                sign = "+" if pct_diff >= 0 else "-"
                diffs.append(f"{sign}{abs(pct_diff):.2f}%")
            else:
                diffs.append("N/A")
        except:
            diffs.append("N/A")
    discount_premium[t] = " \\ ".join(diffs)

# Final DataFrame
df = pd.DataFrame({
    "Ticker": all_tickers,
    "Current Price ($)": [current_prices[t] for t in all_tickers],
    "Historic Prices (3/19 \\ 3/26 \\ 4/4)": [price_snapshots[t] for t in all_tickers],
    "Discount/Premium (3/19 \\ 3/26 \\ 4/4)": [discount_premium[t] for t in all_tickers],
    "10-Day Moving Avg ($)": [round(dma_10[t], 2) if not pd.isna(dma_10[t]) else None for t in all_tickers],
    "52-Week Range": [week_52[t] for t in all_tickers],
    "Analyst Rating": [analyst_rating[t] for t in all_tickers],
    "Target Price ($)": [target_price[t] for t in all_tickers]
})

# Show result
pd.set_option("display.max_rows", None)
print("📊 Portfolio Snapshot —", datetime.today().strftime('%Y-%m-%d'))
df


[*********************100%***********************]  27 of 27 completed
[*********************100%***********************]  27 of 27 completed


📊 Portfolio Snapshot — 2025-05-03


,Ticker,Current Price ($),Historic Prices (3/19 \ 3/26 \ 4/4),Discount/Premium (3/19 \ 3/26 \ 4/4),10-Day Moving Avg ($),52-Week Range,Analyst Rating,Target Price ($)
0,NVDA,114.50,117.52 \ 113.76 \ 94.31,-2.57% \ +0.65% \ +21.41%,106.87,86.62 - 153.13,Strong Buy,163.12236
1,META,597.02,584.06 \ 610.98 \ 504.73,+2.22% \ -2.28% \ +18.29%,540.80,442.65 - 740.91,Strong Buy,705.66565
2,MU,80.72,101.93 \ 92.01 \ 64.72,-20.81% \ -12.27% \ +24.72%,75.80,61.54 - 157.54,Strong Buy,123.77121
3,AMZN,189.98,195.54 \ 201.13 \ 171.0,-2.84% \ -5.54% \ +11.10%,183.63,151.61 - 242.52,Strong Buy,239.53348
4,CRM,275.08,278.95 \ 280.55 \ 240.38,-1.39% \ -1.95% \ +14.43%,260.92,212.0 - 369.0,Strong Buy,364.65070
5,MSFT,435.28,387.82 \ 389.97 \ 359.84,+12.24% \ +11.62% \ +20.96%,392.06,344.79 - 468.35,Strong Buy,506.52500
6,GOOGL,164.03,163.89 \ 165.06 \ 145.6,+0.09% \ -0.62% \ +12.66%,158.06,140.53 - 207.05,Strong Buy,201.74875
7,AVGO,203.64,194.98 \ 179.27 \ 146.29,+4.44% \ +13.59% \ +39.20%,187.02,127.507 - 251.88,Strong Buy,238.54486
8,NOW,977.36,844.12 \ 838.55 \ 721.65,+15.78% \ +16.55% \ +35.43%,898.99,637.99 - 1198.09,Strong Buy,1055.59350
9,AAPL,205.35,215.24 \ 221.53 \ 188.38,-4.59% \ -7.30% \ +9.01%,206.77,169.21 - 260.1,Buy,233.36476
